# DART_API

The Columbia Basin Research website belonging to the University of Washington School of Aquatic & Fishery Sciences provides access to a variety of datasets pertaining to the Columbia River.

The Data Access in Real Time (DART) project has several pages set up to make queries. Although there does not appear to be an API, one of the pages http://www.cbr.washington.edu/dart/query/adult_graph_text will create the URL to download data in csv format.

This notebook builds a simple API-like interface to download data for this project.

 - [Dissecting the URL](#Dissecting-the-URL)
 - [Notebook setup](#Notebook-setup)
 - [Downloading data](#Downloading-data)

## Dissecting the URL


Sockeye daily counts for all locations (Columns) for all days of the year (rows) for 2019

This query selected
 - Output format: CSV
 - Year: 2019
 - Project: BON, JDA, MCN, PRD, RIS, RRH, TDA, WAN, WEL
 - Species: Sockeye
 - 10 Year: No Selection
 - River Data: No Selection

Note: spaces and newlines inserted for readability
http: //www. cbr .washington. edu/dart/cs/php/rpt/mg.php?sc=1&mgconfig=adult

    &outputFormat=csv    # csv - locations in columns, csvSingle = single data point per row
    &year%5B%5D=2019    
    
    &loc%5B%5D=BON&loc%5B%5D=JDA&loc%5B%5D=MCN    
    &loc%5B%5D=PRD&loc%5B%5D=RIS&loc%5B%5D=RRH    
    &loc%5B%5D=TDA&loc%5B%5D=WAN&loc%5B%5D=WEL    
    
    &ftype%5B%5D=fb     
    &data%5B%5D=   
    &data%5B%5D=
    &startdate=1%2F1
    &enddate=12%2F31
    &avgyear=0
    &sumAttribute=none
    &consolidate=1
    &zeros=1
    &grid=1
    &y1min=0
    &y1max=
    &y2min=
    &y2max=
    &size=medium

In [1]:
# Example: 2018, 3 species, all 9 locations
#
# http://www.cbr.washington.edu/dart/cs/php/rpt/mg.php?sc=1&mgconfig=adult&outputFormat=csvSingle
#    &year%5B%5D=2018&loc%5B%5D=BON&loc%5B%5D=JDA&loc%5B%5D=MCN&loc%5B%5D=PRD&loc%5B%5D=RIS&loc%5B%5D=RRH
#    &loc%5B%5D=TDA&loc%5B%5D=WAN&loc%5B%5D=WEL

#    &ftype%5B%5D=fc&ftype%5B%5D=fk&ftype%5B%5D=fb&ftype%5B%5D=fp

#    &data%5B%5D=&data%5B%5D=&startdate=1%2F1&enddate=12%2F31&avgyear=0&sumAttribute=none&consolidate=1&zeros=1&grid=1&y1min=0&y1max=&y2min=&y2max=&size=medium

## Notebook setup

In [2]:
import requests
import csv

import numpy as np
import pandas as pd

In [3]:
# global variables

# DART codes for fish species
species_dict = {'sockeye':'fb',
                'chinook':'fc',
                'coho':'fk',
                'chum':'fe',
                'pink':'fp'}
# DART codes for river data -- only the codes used here
river_data_dict = {
    'Barometric Pressure':'Barometric+Pressure',
    'Dissolved Gas':'Dissolved+Gas',
    'Dissolved Gas Percent':'Dissolved+Gas+Percent',
    'Elevation':'Elevation',    
    'Inflow': 'Inflow',
    'Outflow':'Outflow',
    'Spill':'Spill',
    'Spill Percent':'Spill+Percent', 
    'Temperature (Scroll Case)':'Temp+%28Scroll+Case%29',
    'Temperature (WQM)':'Temp+%28WQM%29',
    'Turbidity':'Turbidity'
}

# Location codes for 9 sites on the Columbia river in Washington State
location_list = ['BON','JDA','MCN','PRD','RIS','RRH','TDA','WAN','WEL']

In [4]:
def construct_query(year,species_list,location_list,riverdata_list,single=True):
    '''Construct a query for a particular year and species of fish. All other options are hard coded.'''
    
    q_base   = 'http://www.cbr.washington.edu/dart/cs/php/rpt/mg.php?sc=1&mgconfig=adult'
    
    # single is single observation per row, as in tidy data. otherwise, wide format
    if (single):
        q_format = '&outputFormat=csvSingle'
    else:
        q_format = '&outputFormat=csv'
    q_year   = '&year%5B%5D=' + str(year)
    
    # select locations (required)
    q_loc = ''
    for location in location_list:
        q_loc  += '&loc%5B%5D=' + location
        #'&loc%5B%5D=RRH&loc%5B%5D=TDA&loc%5B%5D=WAN&loc%5B%5D=WEL'
    
    # select the fish species to retrieve (required)
    q_fish = ''
    for species in species_list:
        q_fish += '&ftype%5B%5D=' + species_dict[species]
    
    # 10 year averages
    q_10yr  = '&data%5B%5D='
    
    # river conditions (optional)
    if (len(riverdata_list)==0):
        q_river = '&data%5B%5D='
    else:
        q_river = ''
        for rd in riverdata_list:
            q_river += '&data%5B%5D=' + river_data_dict[rd]
    
    # optional to set for less than a year; this function defaults to the full year
    q_tail  = '&data%5B%5D=&data%5B%5D=&startdate=1%2F1&enddate=12%2F31&avgyear=0&sumAttribute=none' + \
                '&consolidate=1&zeros=1&grid=1&y1min=0&y1max=&y2min=&y2max=&size=medium'

    
    final_url = q_base + q_format + q_year + q_loc + q_fish + q_10yr + q_river + q_tail
    return final_url

In [5]:
def fetch_data(CSV_URL):
    #https://stackoverflow.com/questions/35371043/use-python-requests-to-download-csv
    #CSV_URL = get_sock

    with requests.Session() as s:
        download = s.get(CSV_URL)

        decoded_content = download.content.decode('utf-8')

        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        #for row in my_list:
            #print(row)
    return my_list

In [6]:
# Example .head() if 'csv' is requested
#     mm/dd  2019:JDA:Sock (fish/day)  2019:BON:Sock (fish/day)
# 0   1/1    0                         0
# 0   1/2    0                         0 


# Example .head() if 'csvSingle' is requested         (also known as 'tidy' data)
#      year   mm-dd   location  parameter  unit       datatype         value
# 0    2019   1-1     JDA       Chin       fish/day   Adult Passage    0
# 1    2019   1-2     JDA       Chin       fish/day   Adult Passage    0

In [7]:
def construct_df(data_as_list, single=True):
    '''Converts the downloaded data to a pandas DataFrame and reformats the columns.'''
    # construct the initial dataframe
    df = pd.DataFrame(data_as_list[2:],columns=data_as_list[1])
    
    if (single):
        # construct a date column
        df['Date'] = df['year'] + '-' + df['mm-dd']  
    else:
        # extract year from column location name ' 2019:MCN:Sock (fish/day)' to '2019'
        year = df.columns[1].split(':')[0].strip()

        # convert column location names from ' 2019:MCN:Sock (fish/day)' to 'MCN'
        locations = [df.columns[i].split(':')[1]  for i in range(1,len(df.columns))]

        # replace the 'mm/dd' column name with 'Date', then convert '1/1' etc. to '1/1/YYYY'
        df.columns = ['Date'] + locations    
        df['Date'] = df['Date'] + '/' + year
    
    # remove the last 7 rows which contain download timestamp and similar info
    df = df[:-7]
    
    return df

## Downloading data

**Example - single species**

In [8]:
# # Example: Single species, single year, all 9 Columbia locations, WIDE format

# my_url = construct_query(2019,['sockeye'],location_list,[],single=False)
# data_as_list = fetch_data(my_url)
# df_sockeye_2019 = construct_df(data_as_list,single=False) # works with non-tidy data

# df_sockeye_2019.head()

In [9]:
# df_sockeye_2019.tail()

**Example - several species**

In [10]:
# # Example: 5 species, single year, all 9 Columbia locations, TIDY format 

# my_url = construct_query(2019,['sockeye','coho','chinook','chum','pink'],location_list,[],single=True)

# data_as_list = fetch_data(my_url)
# df = construct_df(data_as_list,single=True)

# df.head()

In [11]:
# df['parameter'].value_counts()

**Project Data - Fish Counts**

In [12]:
years_of_interest = np.arange(1999,2021,1)
years_of_interest

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [13]:
species_of_interest = ['sockeye','coho','chinook','chum','pink']

In [14]:
df = pd.DataFrame(columns = ['year', 'mm-dd', 'location', 'parameter', 'unit', 'datatype', 'value',
       'Date']) # giant df accummulator

In [15]:
df.shape

(0, 8)

In [16]:
# fetch data for each year
for year in years_of_interest:
    my_url = construct_query(year,species_of_interest,location_list,[],single=True)
    data_as_list = fetch_data(my_url)
    df = df.append(construct_df(data_as_list,single=True),ignore_index=True)

In [17]:
# Save fish data to file
df.to_csv('../data/DART_csv_files/salmon5_20yearsv2.csv',index=False)

In [18]:
df.shape # (292585, 8)

(292585, 8)

**Project Data - Water Conditions**

Water conditions can only be retrieved in addtion to at least one fish; the fish section of the data will then be dropped.

DART has a year-location-parameter limit, so data must be retrieved in smaller pieces.

In [19]:
get_water = 'http://www.cbr.washington.edu/dart/cs/php/rpt/mg.php?sc=1&mgconfig=adult&outputFormat=csvSingle&year%5B%5D=2020&year%5B%5D=2019&year%5B%5D=2018&year%5B%5D=2017&year%5B%5D=2016&year%5B%5D=2015&year%5B%5D=2014&year%5B%5D=2013&year%5B%5D=2012&year%5B%5D=2011&year%5B%5D=2010&year%5B%5D=2009&year%5B%5D=2008&year%5B%5D=2007&year%5B%5D=2006&year%5B%5D=2005&year%5B%5D=2004&year%5B%5D=2003&year%5B%5D=2002&year%5B%5D=2001&year%5B%5D=2000&year%5B%5D=1999&loc%5B%5D=RRH&ftype%5B%5D=fc&data%5B%5D=&data%5B%5D=Dissolved+Gas&data%5B%5D=Dissolved+Gas+Percent&data%5B%5D=Temp+%28Scroll+Case%29&data%5B%5D=Temp+%28WQM%29&data%5B%5D=Turbidity&startdate=1%2F1&enddate=12%2F31&avgyear=0&sumAttribute=none&consolidate=1&zeros=1&grid=1&y1min=0&y1max=&y2min=&y2max=&size=medium'

In [20]:
data_as_list = fetch_data(get_water)

In [21]:
data_as_list[:5]

[[],
 ['year', 'mm-dd', 'location', 'parameter', 'unit', 'datatype', 'value'],
 ['2020', '1-1', 'RRH', 'Chin', 'fish/day', 'Adult Passage', 'NA'],
 ['2020', '1-2', 'RRH', 'Chin', 'fish/day', 'Adult Passage', 'NA'],
 ['2020', '1-3', 'RRH', 'Chin', 'fish/day', 'Adult Passage', 'NA']]

In [22]:
df_water = construct_df(data_as_list,single=True)

In [23]:
df_water.shape

(45384, 8)

In [24]:
df_water['parameter'].value_counts()

tempc      8052
gaspct     8052
Chin       8052
disgas     8052
turb       7686
tempscr    5490
Name: parameter, dtype: int64

In [25]:
# drop the fish
df_water = df_water[df_water['parameter']!='Chin']

In [26]:
df_water.shape

(37332, 8)

In [27]:
river_data_list = ['Barometric Pressure',
    'Dissolved Gas',
    'Dissolved Gas Percent',
    'Elevation',    
    'Inflow',
    'Outflow',
    'Spill',
    'Spill Percent', 
    'Temperature (Scroll Case)',
    'Temperature (WQM)',
    'Turbidity']

In [28]:
df_water = pd.DataFrame(columns = ['year', 'mm-dd', 'location', 'parameter', 'unit', 'datatype', 'value',
       'Date']) # giant df accummulator

In [29]:
location_list # WAN doesn't start until 2006!

['BON', 'JDA', 'MCN', 'PRD', 'RIS', 'RRH', 'TDA', 'WAN', 'WEL']

In [32]:
# fetch data for each year
for year in years_of_interest:
    for location in location_list:
        # Skip WAN until 2006
        if ((year < 2006) and (location == 'WAN')):
            print("Skipping WAN prior to 2006")
            break
        my_url = construct_query(year,['chinook'],[location],river_data_list,single=True)
        data_as_list = fetch_data(my_url)
        df_water = df_water.append(construct_df(data_as_list,single=True),ignore_index=True)
        print(f"Completed: {year} {location}")

Completed: 1999 BON
Completed: 1999 JDA
Completed: 1999 MCN
Completed: 1999 PRD
Completed: 1999 RIS
Completed: 1999 RRH
Completed: 1999 TDA
Skipping WAN prior to 2006
Completed: 2000 BON
Completed: 2000 JDA
Completed: 2000 MCN
Completed: 2000 PRD
Completed: 2000 RIS
Completed: 2000 RRH
Completed: 2000 TDA
Skipping WAN prior to 2006
Completed: 2001 BON
Completed: 2001 JDA
Completed: 2001 MCN
Completed: 2001 PRD
Completed: 2001 RIS
Completed: 2001 RRH
Completed: 2001 TDA
Skipping WAN prior to 2006
Completed: 2002 BON
Completed: 2002 JDA
Completed: 2002 MCN
Completed: 2002 PRD
Completed: 2002 RIS
Completed: 2002 RRH
Completed: 2002 TDA
Skipping WAN prior to 2006
Completed: 2003 BON
Completed: 2003 JDA
Completed: 2003 MCN
Completed: 2003 PRD
Completed: 2003 RIS
Completed: 2003 RRH
Completed: 2003 TDA
Skipping WAN prior to 2006
Completed: 2004 BON
Completed: 2004 JDA
Completed: 2004 MCN
Completed: 2004 PRD
Completed: 2004 RIS
Completed: 2004 RRH
Completed: 2004 TDA
Skipping WAN prior to 200

In [33]:
df_water.head(2)

,year,mm-dd,location,parameter,unit,datatype,value,Date
0,1999,1-1,BON,Chin,fish/day,Adult Passage,NA,1999-1-1
1,1999,1-2,BON,Chin,fish/day,Adult Passage,NA,1999-1-2


In [34]:
df_water.shape

(734560, 8)

In [35]:
# drop the fish
df_water = df_water[df_water['parameter']!='Chin']

In [36]:
df_water.shape

(667350, 8)

In [37]:
# Save fish data to file
df_water.to_csv('../data/DART_csv_files/riverdata_20years.csv',index=False)